##### import

In [1]:
import os
from os import path as P
import sys
sys.path.append(P.abspath(P.join(os.getcwd(), '..', 'lib')))

In [2]:
from nn_template import Cfg
import nn_template.data_augmentation
import nn_template.datasets
import nn_template.hyperparameters_tuning.optuna
from nn_template.callbacks.log_artifacts import Export2DLabel

In [3]:
import matplotlib.pyplot as plt

### Load Dataset

In [4]:
cfg = Cfg.Parser('vessels.yaml').get_config()

AssertionError: Error when parsing None: __m must be a Mapping, not <class 'bool'>

In [ ]:
train = cfg['datasets.train'].dataset()
validate = cfg['datasets.validate'].dataset()
test = cfg['datasets.test.DRIVE'].dataset()

In [ ]:
%timeit train[1]
data = [train[1] for i in range(3)]

In [ ]:
plt.rcParams["figure.figsize"] = (25, 25)

In [ ]:
fig, axs = plt.subplots(len(data),3)
for i, d in enumerate(data):
    axs[i, 0].imshow(d['x'].permute((1,2,0)).flip(2))
    axs[i, 1].imshow(d['y'], cmap='gray', vmax=1)
    axs[i, 2].imshow(d['mask'], cmap='gray')

## Test Dataloader

In [ ]:
datasets = cfg['datasets']
train_dataloader, val_dataloader = datasets.create_train_val_dataloaders()
test_dataloader = datasets.create_test_dataloaders()

In [ ]:
batch = next(iter(test_dataloader[0]))
fig, axs = plt.subplots(len(batch['x']),3)
for i, (x, y, mask) in enumerate(zip(*[batch[_] for _ in 'x y mask'.split()])):
    axs[i, 0].imshow(x.permute((1,2,0)).flip(2))
    axs[i, 1].imshow(y, cmap='gray', vmax=1)
    axs[i, 2].imshow(mask, cmap='gray')

In [ ]:
cmap_vessel = {(0, 0): 'black', (1, 1): 'white', (1, 0): 'orange', (0, 1): 'greenyellow', 'default': 'lightgray'}
formatter = Export2DLabel(cmap_vessel)
formatter.format_batch(batch)